# Training a Regression Network on Radar Dataset (Ouachita Mountains)

## Importing the required modules

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(suppress=True)
import scipy.io as sio
import pandas as pd
import os
import re
import math
import itertools
import time
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import KFold
from itertools import product
from sklearn.utils.class_weight import compute_class_weight
torch.cuda.empty_cache()

# Reading the labels and creating train and test data

In [2]:
def index(string):
    s = re.findall("[0-9]", string)
    return int(''.join(s))

scenario_idx = 62 # Ouachita Mountains
names = os.listdir(f'num{scenario_idx}_NAMF_DATA_20_rng30_pulse1_1000_100k/')
names = sorted(names, key = index)
print(len(names))
y = pd.read_csv(f'num{scenario_idx}_Ground_Truth_20_rng30_1000_100k_m.csv')
col_names = y.columns[4:7]
y = y[col_names].to_numpy()

y_train = y[:int(0.9*len(names))]
y_test = y[(int(0.9*len(names))+1):]
training_names = names[:int(0.9*len(names))]
test_names = names[(int(0.9*len(names))+1):]

print('Training labels: ')
print(y_train)

# Tensor Corners
##################################################################################
# num29: [10851, 215, -5.45], num60: [11073, 215, -5.3], num62: [11471, 215, -5.6]
# num76: [11388, 215, -6.15], num35: [11381, 215, -0.95]
##################################################################################

# Training dataset global constants
coord_tr = [11471, 215, -5.6] # Tensor corner
rng_res_tr = 59.9585/2         # Range resolution
az_step_tr = 0.4               # Azimuth step size
el_step_tr = 0.01              # Elevation step size

# Test dataset global constants
coord_ts = [11471, 215, -5.6] # Tensor corner
rng_res_ts = 59.9585/2         # Range resolution
az_step_ts = 0.4               # Azimuth step size
el_step_ts = 0.01              # Elevation step size


def Drawing_Batch(names, label, bs, ind, normalize = True):
    x = []
    labels = []
    
    for j in range(ind*bs, (ind+1)*bs):
        try: temp = sio.loadmat(f'num{scenario_idx}_NAMF_DATA_20_rng30_pulse1_1000_100k/'+names[j])['P']
        except: break
        if normalize:
            Anorm = temp - np.min(temp.flatten())
            temp = np.divide(Anorm, np.max(Anorm.flatten()))
        x.append(temp)
        labels.append(label[j,:])
        
    x = torch.FloatTensor(np.array(x))
    labels = torch.FloatTensor(np.array(labels))
    return x,labels

100000
Training labels: 
[[ 0.19278 19.175    5.3732 ]
 [11.244   23.269   24.522  ]
 [-0.2566  20.21     9.9473 ]
 ...
 [ 7.7966  22.974   29.859  ]
 [15.579   24.133   17.291  ]
 [ 6.6286  23.169   29.05   ]]


In [3]:
print(len(y_train))
print(len(y_test))

90000
9999


## Define a regression CNN and instantiating it

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv1d(21, 32, 3, 1)
        self.conv2 = nn.Conv1d(32, 64, 3, 1)
        self.batchnorm1 = nn.BatchNorm1d(32)
        self.batchnorm2 = nn.BatchNorm1d(64)
        self.fc1 = nn.Linear(64 * 5, 20)  # Adjust input size based on the output of conv layers and max pooling
        self.fc2_reg = nn.Linear(20, 2)  # Adjusted output size

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(self.batchnorm1(x))
        x = F.max_pool1d(x, 2)
        
        x = self.conv2(x)
        x = F.relu(self.batchnorm2(x))
        x = F.max_pool1d(x, 2)
        
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        output_reg = self.fc2_reg(x)  # (bs, 2)
        
        return output_reg
    
from torchsummary import summary
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
model = Net()
model = model.to(device)
if device == 'cuda:0':
    model = torch.nn.DataParallel(model)
    cudnn.benchmark = True
print(summary(model,(21,26)))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1               [-1, 32, 24]           2,048
       BatchNorm1d-2               [-1, 32, 24]              64
            Conv1d-3               [-1, 64, 10]           6,208
       BatchNorm1d-4               [-1, 64, 10]             128
            Linear-5                   [-1, 20]           6,420
            Linear-6                    [-1, 2]              42
               Net-7                    [-1, 2]               0
Total params: 14,910
Trainable params: 14,910
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.02
Params size (MB): 0.06
Estimated Total Size (MB): 0.08
----------------------------------------------------------------
None


## Global Definitions

In [6]:
bs = 128  # batch_size
num_epoch = 50  # number of epochs
PATH = './ckpt_model.pth'   # forsaving the model
criterion = nn.MSELoss()
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

# Define a Loss function and optimizer; Using GPU or CPU
model = Net()
optimizer = optim.Adam(model.parameters(), lr = 0.001)
model = model.to(device)
if device == 'cuda:0':
    model = torch.nn.DataParallel(model)
    cudnn.benchmark = True
    
def Spher2Cart_1D(spherical):
    cartesian = np.zeros(3)
    hypotenuse = np.cos(np.radians(spherical[2]))*spherical[0]
    cartesian[0] = np.cos(np.radians(spherical[1]))*hypotenuse
    cartesian[1] = -np.sin(np.radians(spherical[1]))*hypotenuse
    cartesian[2] = np.sin(np.radians(spherical[2]))*spherical[0]
    return cartesian

## Train and evaluate the network

In [7]:
def main(training_names, test_names, bs, num_epoch, y_train, y_test, normalize=True):
    best_error = 1e+20      # a dummy and very large number for saving the best discovered model
    for epoch in range(num_epoch):
        print('Epoch {}/{}'.format(epoch, num_epoch))
        print('-'*10)
        running_loss_train = 0
        running_loss_test = 0

        model.train()
        for i in range(0, len(training_names)//bs):
            x_train, labels = Drawing_Batch(training_names, y_train, bs, i, normalize)
            x_train = x_train.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            out = model(x_train)
            loss = criterion(out, labels[:,0:2])
            loss.backward()
            optimizer.step()
            running_loss_train += loss.item()
            
        out = torch.cat((out, torch.unsqueeze(labels[:,2], dim=1)), dim=1)
        
        true_train = Spher2Cart_1D(np.multiply(labels.cpu().data.numpy()[1,], [rng_res_tr,az_step_tr,el_step_tr]) + coord_tr)
        pred_train = Spher2Cart_1D(np.multiply(out.cpu().data.numpy()[1,], [rng_res_tr,az_step_tr,el_step_tr]) + coord_tr)
        print(true_train)
        print(pred_train)

        model.eval()
        with torch.no_grad():
            for i in range(0, len(test_names)//bs):
                x_test, labels_test = Drawing_Batch(test_names, y_test, bs, i, normalize)
                x_test = x_test.to(device)
                labels_test = labels_test.to(device)
                out_test = model(x_test)
                loss_test = criterion(out_test, labels_test[:,0:2])
                running_loss_test += loss_test.item()

        out_test = torch.cat((out_test, torch.unsqueeze(labels_test[:,2], dim=1)), dim=1)
        
        true_test = Spher2Cart_1D(np.multiply(labels_test.cpu().data.numpy()[1,], [rng_res_ts,az_step_ts,el_step_ts]) + coord_ts)
        pred_test = Spher2Cart_1D(np.multiply(out_test.cpu().data.numpy()[1,], [rng_res_ts,az_step_ts,el_step_ts]) + coord_ts)
        print(true_test)
        print(pred_test)
        
        epoch_loss_train = running_loss_train*x_train.size()[0]/len(training_names)
        epoch_loss_test = running_loss_test*x_test.size()[0]/len(test_names)

        print('Train Loss: {:.6f} ---- Test Loss: {:.6f}'.format(epoch_loss_train, epoch_loss_test))
        if epoch%5==0:
            if epoch_loss_test < best_error:
                torch.save(model.state_dict(), PATH)
                best_accuracy = epoch_loss_test

start = time.time()
main(training_names, test_names, bs, num_epoch, y_train, y_test, normalize=True)
end = time.time()
print(end - start)

Epoch 0/50
----------
[-8421.63922973  7801.90963099 -1096.1574508 ]
[-8438.09755764  7782.44005031 -1096.04958087]
[-8443.49210233  8155.83627122 -1085.60648848]
[-8294.314676    8223.14936929 -1080.10036779]
Train Loss: 13.659371 ---- Test Loss: 3.913199
Epoch 1/50
----------
[-8421.63922973  7801.90963099 -1096.1574508 ]
[-8433.84037299  7782.90691995 -1095.78104197]
[-8443.49210233  8155.83627122 -1085.60648848]
[-8303.72121139  8223.17797002 -1080.72014779]
Train Loss: 3.602420 ---- Test Loss: 3.663542
Epoch 2/50
----------
[-8421.63922973  7801.90963099 -1096.1574508 ]
[-8434.04400169  7783.11541896 -1095.80883185]
[-8443.49210233  8155.83627122 -1085.60648848]
[-8323.63756614  8205.65731184 -1080.89150315]
Train Loss: 3.522377 ---- Test Loss: 3.913097
Epoch 3/50
----------
[-8421.63922973  7801.90963099 -1096.1574508 ]
[-8428.9360065   7787.54880392 -1095.73767279]
[-8443.49210233  8155.83627122 -1085.60648848]
[-8332.6674182   8190.10949616 -1080.47796984]
Train Loss: 3.494644 

[-8421.63922973  7801.90963099 -1096.1574508 ]
[-8425.97754445  7795.11238546 -1096.02051375]
[-8443.49210233  8155.83627122 -1085.60648848]
[-8351.46138263  8198.12850036 -1082.23752479]
Train Loss: 3.448464 ---- Test Loss: 3.697425
Epoch 33/50
----------
[-8421.63922973  7801.90963099 -1096.1574508 ]
[-8424.20934043  7796.93642414 -1096.01488199]
[-8443.49210233  8155.83627122 -1085.60648848]
[-8354.19787443  8197.26194958 -1082.36200471]
Train Loss: 3.447379 ---- Test Loss: 3.699110
Epoch 34/50
----------
[-8421.63922973  7801.90963099 -1096.1574508 ]
[-8426.30498869  7794.55431201 -1096.00727934]
[-8443.49210233  8155.83627122 -1085.60648848]
[-8350.82731125  8197.05846446 -1082.12636114]
Train Loss: 3.445119 ---- Test Loss: 3.661107
Epoch 35/50
----------
[-8421.63922973  7801.90963099 -1096.1574508 ]
[-8424.17765425  7796.52889449 -1095.98623066]
[-8443.49210233  8155.83627122 -1085.60648848]
[-8350.69991333  8201.38361503 -1082.39817817]
Train Loss: 3.443345 ---- Test Loss: 3.66

In [8]:
def Spher2Cart_2D(spherical):
    cartesian = np.zeros((len(spherical),3))
    hypotenuse = np.multiply(np.cos(np.radians(spherical[:,2])), spherical[:,0])
    cartesian[:,0] = np.multiply(np.cos(np.radians(spherical[:,1])), hypotenuse)
    cartesian[:,1] = np.multiply(-np.sin(np.radians(spherical[:,1])), hypotenuse)
    cartesian[:,2] = np.multiply(np.sin(np.radians(spherical[:,2])), spherical[:,0])
    return cartesian

# Testing: (range,az,el)
model.eval()
out_test_reg = np.zeros((len(y_test),3))
labels_test_reg = np.zeros((len(y_test),3))

for i in range(0, len(y_test)//bs):  
    x_test, labels_test = Drawing_Batch(test_names, y_test, bs, i, True)
    labels_test = labels_test.cpu().data.numpy()
    labels_test_reg[bs*i : bs*i + bs] = (labels_test[:,0:3])

    cur_test_reg = model(x_test)
    out_test_reg[bs*i : bs*i + bs, 0:2] = cur_test_reg.cpu().data.numpy()
    out_test_reg[bs*i : bs*i + bs, 2] = labels_test_reg[bs*i : bs*i + bs, 2]
    
azim_tot = 0
for i in np.arange(0,len(out_test_reg),1):
    azim_tot += np.linalg.norm(out_test_reg[i,1] - labels_test_reg[i,1])
    
azim_tot = azim_tot / len(out_test_reg)
print(f'Azimuth Estimation Error (deg) = {azim_tot}')

new_data = Spher2Cart_2D(np.multiply(out_test_reg, [rng_res_ts,az_step_ts,el_step_ts]) + coord_ts)
new_labels_data = Spher2Cart_2D(np.multiply(labels_test_reg, [rng_res_ts,az_step_ts,el_step_ts]) + coord_ts)

sum_tot = 0
for i in np.arange(0,len(new_data) - (len(new_data) % bs),1):
    sum_tot += np.linalg.norm(new_data[i,:] - new_labels_data[i,:])
    
# def reject_outliers(data, m=2):
#     return data[abs(data - np.mean(data)) < m * np.std(data)]
# 
# azims = (np.multiply(out_test_reg, [rng_res_ts,az_step_ts,el_step_ts]) - np.multiply(labels_test_reg, [rng_res_ts,az_step_ts,el_step_ts]))[:,1]
# azims = reject_outliers(azims)
# mse = np.mean(azims**2)
# bias_sq = np.mean(azims)**2
# var = np.var(azims)
# print(mse, bias_sq, var)

sum_tot = sum_tot / (len(new_data) - (len(new_data) % bs))
print(f'Localization Error (m) = {sum_tot}')

Azimuth Estimation Error (deg) = 0.4839181860813446
Localization Error (m) = 68.31161613254166
